In [1]:
import pandas as pd
import sklearn.model_selection as ms
from sklearn.ensemble import GradientBoostingRegressor
import sklearn.preprocessing as pre
import sklearn.metrics as me
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
import numpy as np
import xgboost as xgb

import pickle

In [2]:
RANDOM_STATE = 42

In [3]:
submission = pd.read_csv('../Data/files/stats_test.csv')
flights_engi = pd.read_csv('../Data/files/stats.csv')
flights_engi.head()

,branded_code_share,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,origin_airport_id_mean,origin_airport_id_median,dest_airport_id_mean,dest_airport_id_median,...,day_of_week_median,week_of_year_mean,week_of_year_median,tail_num_mean,tail_num_median,op_unique_carrier_mean,op_unique_carrier_median,mkt_unique_carrier_mean,mkt_unique_carrier_median,stats_pred
0,0,1000,1835,335.0,2565,87.0,6.297634,-6.0,18.862792,-3.0,...,-5.0,5.192664,-6.0,-1.452778,-5.0,0.367058,-6.0,0.645435,-6.0,0.599476
1,1,1345,1424,99.0,500,-3.0,2.734614,-7.0,4.033033,-6.0,...,-6.0,2.912060,-6.0,-0.328512,-11.0,3.516933,-10.0,2.421644,-8.0,0.733710
2,1,1220,1405,105.0,391,3.0,7.672142,-5.0,1.498925,-7.0,...,-7.0,5.981355,-6.0,0.955516,-7.5,7.113200,-7.0,2.421644,-8.0,0.674812
3,0,2120,2340,80.0,451,-4.0,2.127281,-7.0,6.327156,-4.0,...,-6.0,6.975293,-6.0,5.558870,-5.0,3.577572,-5.0,3.577572,-5.0,0.552997
4,0,1130,1730,240.0,1620,7.0,7.672142,-5.0,10.926134,-6.0,...,-5.0,9.774019,-4.0,-4.348974,-11.0,0.449845,-8.0,2.421644,-8.0,0.658946


In [4]:
submission.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin_airport_id,origin,dest_airport_id,...,day_of_week_median,week_of_year_mean,week_of_year_median,tail_num_mean,tail_num_median,op_unique_carrier_mean,op_unique_carrier_median,mkt_unique_carrier_mean,mkt_unique_carrier_median,stats_pred
0,2020-01-01 00:00:00,WN,0,WN,5888,WN,N951WN,13891,ONT,14771,...,-6.0,6.771881,-5.0,3.229261,-6.0,3.577572,-5.0,3.577572,-5.0,0.591202
1,2020-01-01 00:00:00,WN,0,WN,6276,WN,N467WN,13891,ONT,14771,...,-6.0,6.771881,-5.0,1.888704,-5.0,3.577572,-5.0,3.577572,-5.0,0.591202
2,2020-01-01 00:00:00,WN,0,WN,4598,WN,N7885A,13891,ONT,14831,...,-6.0,6.771881,-5.0,3.958029,-5.0,3.577572,-5.0,3.577572,-5.0,0.591202
3,2020-01-01 00:00:00,WN,0,WN,4761,WN,N551WN,13891,ONT,14831,...,-6.0,6.771881,-5.0,4.577049,-4.0,3.577572,-5.0,3.577572,-5.0,0.591202
4,2020-01-01 00:00:00,WN,0,WN,5162,WN,N968WN,13891,ONT,14831,...,-6.0,6.771881,-5.0,2.912480,-5.0,3.577572,-5.0,3.577572,-5.0,0.591202


In [5]:
submission = submission.drop('branded_code_share', axis=1)

flights_engi = flights_engi.drop(columns=['branded_code_share'])

In [6]:
cols_to_test = (set(flights_engi.columns) - {'arr_delay'})

# preview
flights_engi.columns

Index(['crs_dep_time', 'crs_arr_time', 'crs_elapsed_time', 'distance',
       'arr_delay', 'origin_airport_id_mean', 'origin_airport_id_median',
       'dest_airport_id_mean', 'dest_airport_id_median', 'month_of_year_mean',
       'month_of_year_median', 'day_of_week_mean', 'day_of_week_median',
       'week_of_year_mean', 'week_of_year_median', 'tail_num_mean',
       'tail_num_median', 'op_unique_carrier_mean', 'op_unique_carrier_median',
       'mkt_unique_carrier_mean', 'mkt_unique_carrier_median', 'stats_pred'],
      dtype='object')

In [7]:
X = flights_engi.drop('arr_delay', axis=1)
y = flights_engi['arr_delay']

### Processing the data

In [8]:
X_train, X_test, y_train, y_test = ms.train_test_split(X, y, random_state=RANDOM_STATE, test_size=0.3)

In [9]:
# pipe = Pipeline(steps=[
#     # ('poly', pre.PolynomialFeatures()),
#     ('std_scaler', pre.StandardScaler()),
#     ('pca', PCA(random_state=RANDOM_STATE)),
#     ('gbr', xgb.XGBRegressor(random_state=RANDOM_STATE),
#                                  ),
#     ])
gbr = xgb.XGBRegressor(random_state=RANDOM_STATE)

In [10]:
gbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [11]:
# hyperparameters = {
#     'gbr__learning_rate': np.arange(0.1, 1, 0.3),
#     'gbr__n_estimators': np.arange(100, 1000, 400)
# }

In [12]:
# grid_search = ms.GridSearchCV(pipe, hyperparameters, verbose=1, n_jobs=4)

In [13]:
# grid_search.fit(X_train, y_train)

In [14]:
# model = grid_search.best_estimator_
# model

In [15]:
y_pred = gbr.predict(X_test)

In [16]:
me.mean_squared_error(y_test, y_pred)

2355.2049032966

In [17]:
me.r2_score(y_test, y_pred)

0.04189669784277095

In [18]:
# Save the model
# pickle.dump(model, open('first_submission_gbr.sav', 'wb'))

There is a lot more to iterate! See the repo readme for future directions.

### Get submission predictions

In [19]:
predicted_delay = gbr.predict(submission[cols_to_test])
submission['predicted_delay'] = predicted_delay

In [20]:
cols_to_submit = ['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest', 'predicted_delay']
submission.columns

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier', 'mkt_carrier_fl_num',
       'op_unique_carrier', 'tail_num', 'origin_airport_id', 'origin',
       'dest_airport_id', 'dest', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance', 'day_of_week', 'month_of_year',
       'week_of_year', 'origin_airport_id_mean', 'origin_airport_id_median',
       'dest_airport_id_mean', 'dest_airport_id_median', 'month_of_year_mean',
       'month_of_year_median', 'day_of_week_mean', 'day_of_week_median',
       'week_of_year_mean', 'week_of_year_median', 'tail_num_mean',
       'tail_num_median', 'op_unique_carrier_mean', 'op_unique_carrier_median',
       'mkt_unique_carrier_mean', 'mkt_unique_carrier_median', 'stats_pred',
       'predicted_delay'],
      dtype='object')

In [28]:
submission['fl_date']

0         2020-01-01
1         2020-01-01
2         2020-01-01
3         2020-01-01
4         2020-01-01
             ...    
150618    2020-01-07
150619    2020-01-07
150620    2020-01-07
150621    2020-01-07
150622    2020-01-07
Name: fl_date, Length: 150623, dtype: object

In [29]:
# Save the predictions for submission

submission[cols_to_submit].to_csv('final_submission.csv')